In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
import sklearn.metrics
import scipy as sp
from scipy import sparse
from sklearn.linear_model import LogisticRegression

# 1. Sparse logistics classifier

In [2]:
features = pd.read_csv("features.csv")
features_check = pd.read_csv("features_test.csv")
name_vector = list(features_check.columns)[2:]
features = features.fillna(0)
features_check = features_check.fillna(0)
data = features
sub = features_check
y = data['radiant_win']
del(features)
del(features_check)

In [3]:
num_heroes = len(data['r1_hero'].unique()) 
r_new = []
for i in xrange(num_heroes):
    r_new += ['r'+str(i+1)+'hero','r'+str(i+1)+'level', 'r'+str(i+1)+'xp', 'r'+str(i+1)+'gold', 'r'+str(i+1)+'lh',
             'r'+str(i+1)+'kills','r'+str(i+1)+'deaths','r'+str(i+1)+'items', 'd'+str(i+1)+'hero', 'd'+str(i+1)+'level', 
             'd'+str(i+1)+'xp', 'd'+str(i+1)+'gold', 'd'+str(i+1)+'lh','d'+str(i+1)+'kills','d'+str(i+1)+'deaths',
             'd'+str(i+1)+'items']

In [4]:
data=data.fillna(0)
r1 = data['r1_hero']*16-16
r2 = data['r2_hero']*16-16
r3 = data['r3_hero']*16-16
r4 = data['r4_hero']*16-16
r5 = data['r5_hero']*16-16
d1 = data['d1_hero']*16-16
d2 = data['d2_hero']*16-16
d3 = data['d3_hero']*16-16
d4 = data['d4_hero']*16-16
d5 = data['d5_hero']*16-16
r = np.transpose([r1,r2,r3,r4,r5,d1,d2,d3,d4,d5])
names = list(data.columns.values)
row = []
col = []
val = []
for i in xrange(len(data)):
    for k in xrange(5):
        for j in xrange(8):
            row.insert(j+8*k+i*1000,i)
            col.insert(j+8*k+i*1000,r[i,k]+j)
            val.insert(j+8*k+i*1000,data[names[3+j+8*k]][i])
            row.insert(j+8*(k+5)+i*1000,i)
            col.insert(j+8*(k+5)+i*1000,r[i,(k+5)]+j)
            val.insert(j+8*(k+5)+i*1000,-data[names[3+j+8*(k+5)]][i])

In [5]:
sub=sub.fillna(0)
r1 = sub['r1_hero']*16-16
r2 = sub['r2_hero']*16-16
r3 = sub['r3_hero']*16-16
r4 = sub['r4_hero']*16-16
r5 = sub['r5_hero']*16-16
d1 = sub['d1_hero']*16-16
d2 = sub['d2_hero']*16-16
d3 = sub['d3_hero']*16-16
d4 = sub['d4_hero']*16-16
d5 = sub['d5_hero']*16-16
r = np.transpose([r1,r2,r3,r4,r5,d1,d2,d3,d4,d5])
names = list(data.columns.values)
row_s = []
col_s = []
val_s = []
for i in xrange(len(sub)):
    for k in xrange(5):
        for j in xrange(8):
            row_s.insert(j+8*k+i*1000,i)
            col_s.insert(j+8*k+i*1000,r[i,k]+j)
            val_s.insert(j+8*k+i*1000,sub[names[3+j+8*k]][i])
            row_s.insert(j+8*(k+5)+i*1000,i)
            col_s.insert(j+8*(k+5)+i*1000,r[i,(k+5)]+j)
            val_s.insert(j+8*(k+5)+i*1000,-sub[names[3+j+8*(k+5)]][i])

In [6]:
nele=len(data)
nbus=np.amax(r)+8
sp_features = sparse.coo_matrix((val, (row,col)), shape=(nele, nbus), dtype=float)

In [7]:
nele=len(sub)
nbus=np.amax(r)+8
sub_features = sparse.coo_matrix((val_s, (row_s,col_s)), shape=(nele, nbus), dtype=float)

In [8]:
sp_csr = sp_features.tocsr()
del(sp_features)
del val, row, col

In [9]:
sub_csr = sub_features.tocsr()
del(sub_features)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
...     sp_csr, y, test_size=0.3, random_state=1)
del(sp_csr)

In [11]:
model1 = LogisticRegression(C=0.017, penalty='l1', tol=0.01, random_state=142)

In [12]:
model1.fit(X_train,y_train)

LogisticRegression(C=0.017, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=142, solver='liblinear', tol=0.01,
          verbose=0, warm_start=False)

In [13]:
pred1 = model1.predict_proba(X_test.toarray())[:,1]
print roc_auc_score(y_test, pred1)

0.75556750871


In [ ]:
a = np.ones(len(data))/len(data)

# Roles logistics classifier

In [14]:
features = pd.read_csv("features.csv")
features_check = pd.read_csv("features_test.csv")
name_vector = features_check.columns
features = features.fillna(0)
y = features['radiant_win']
data=features[name_vector]
del features
del features_check

In [15]:
heroes = pd.read_csv('dictionaries/heroes2.csv', ';')
type_vector = list(heroes.columns)[3:]
heroes['sum']=1.*heroes[type_vector].sum(axis=1)
for i in xrange(len(type_vector)):
    heroes[type_vector[i]]=np.divide(heroes[type_vector[i]],heroes['sum'])
del heroes['sum']
new = []
r_old = []
d_old = []
type_vector = list(heroes.columns)[3:]
attr = ['_hero', '_level', '_xp', '_gold', '_lh', '_kills', '_deaths', '_items']
for k in type_vector:
    for i in attr:
        new += [k+i]
for k in range(0,5):
    for i in attr:
        r_old+=['r'+str(k+1)+i] 
for k in range(0,5):
    for i in attr:
        d_old+=['d'+str(k+1)+i] 
for i in list(new):
    data[i]=0
    sub[i]=0
type_vector = list(heroes.columns)[3:12]
type_vector.append('match_id')

In [16]:
for j in xrange(5):
    R = pd.merge(data[['match_id',r_old[8*j]]],heroes,left_on=r_old[8*j],right_on='id').sort("match_id")
    R.index = range(0,len(R))
    D = pd.merge(data[['match_id',d_old[8*j]]],heroes,left_on=d_old[8*j],right_on='id').sort("match_id")
    D.index = range(0,len(D))
    for i in xrange(9):
        for k in xrange(8):
            x=type_vector[i]
            if k == 0:
                data[new[8*i+k]]+=R[x]
                data[new[8*i+k]]-=D[x]
            else:
                data[new[8*i+k]]+=R[x]*data[r_old[8*j+k]]
                data[new[8*i+k]]-=D[x]*data[d_old[8*j+k]] 
    R = pd.merge(sub[['match_id',r_old[8*j]]],heroes,left_on=r_old[8*j],right_on='id').sort("match_id")
    R.index = range(0,len(R))
    D = pd.merge(sub[['match_id',d_old[8*j]]],heroes,left_on=d_old[8*j],right_on='id').sort("match_id")
    D.index = range(0,len(D))
    for i in xrange(9):
        for k in xrange(8):
            x=type_vector[i]
            if k == 0:
                sub[new[8*i+k]]+=R[x]
                sub[new[8*i+k]]-=D[x]
            else:
                sub[new[8*i+k]]+=R[x]*sub[r_old[8*j+k]]
                sub[new[8*i+k]]-=D[x]*sub[d_old[8*j+k]]  

/opt/anaconda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app
/opt/anaconda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/opt/anaconda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:15: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/opt/anaconda/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [31]:
data[data.columns[83:102]].head()

,first_blood_time,first_blood_team,first_blood_player1,first_blood_player2,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_boots_count,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count
0,7,1,9,0,134,-80,244,2,2,2,0,35,103,-84,221,3,4,2,2
1,54,1,7,0,173,-80,0,2,0,2,0,-20,149,-84,195,5,4,3,1
2,224,0,3,0,63,-82,0,2,5,2,1,-39,45,-77,221,3,4,3,1
3,0,0,0,0,208,-75,0,0,3,2,0,-30,124,-80,184,0,4,2,0
4,-21,1,6,0,166,-81,181,1,4,2,0,46,182,-80,225,6,3,3,0


In [26]:
sub.shape

(17177, 175)

### Roles classifier

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
...     data[data.columns[103:]], y, test_size=0.3, random_state=1)

In [28]:
model1 = LogisticRegression(C=0.017, penalty='l1', tol=0.01, random_state=142)

In [29]:
model1.fit(X_train,y_train)

LogisticRegression(C=0.017, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=142, solver='liblinear', tol=0.01,
          verbose=0, warm_start=False)

In [30]:
pred1 = model1.predict_proba(X_test)[:,1]
print roc_auc_score(y_test, pred1)

0.725115997473


### Parameters classifier

In [32]:
X_train, X_test, y_train, y_test = train_test_split(
...     data[data.columns[83:102]], y, test_size=0.3, random_state=1)

In [33]:
model1 = LogisticRegression(C=0.017, penalty='l1', tol=0.01, random_state=142)

In [34]:
model1.fit(X_train,y_train)

LogisticRegression(C=0.017, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=142, solver='liblinear', tol=0.01,
          verbose=0, warm_start=False)

In [35]:
pred1 = model1.predict_proba(X_test)[:,1]
print roc_auc_score(y_test, pred1)

0.623421043525


### Row players data classifier

In [36]:
X_train, X_test, y_train, y_test = train_test_split(
...     data[data.columns[:82]], y, test_size=0.3, random_state=1)

In [37]:
model1 = LogisticRegression(C=0.017, penalty='l1', tol=0.01, random_state=142)

In [38]:
model1.fit(X_train,y_train)

LogisticRegression(C=0.017, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=142, solver='liblinear', tol=0.01,
          verbose=0, warm_start=False)

In [39]:
pred1 = model1.predict_proba(X_test)[:,1]
print roc_auc_score(y_test, pred1)

0.714368836246


In [ ]:
del(data)

# 7. Gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

In [ ]:
model2 = GradientBoostingClassifier(n_estimators=300, verbose=True, random_state=241, learning_rate=0.3)
model2.fit(X_train,y_train)

In [ ]:
pred1 = model1.predict_proba(X_test.toarray())[:, 1]
pred2 = model2.predict_proba(X_test.toarray())[:, 1]
print roc_auc_score(y_test, pred1)
print roc_auc_score(y_test, pred2)